# Выполнил: Карташов ДД из ИМ17-06Б

In [1]:
import numpy as np
import copy

## структуры данных для представления графов

In [2]:
class container:
    def __init__(self,arr = []):
        try:
            self.arr = copy.copy(arr.arr)
        except:
            self.arr = copy.copy(arr)
    def push(self,elem): pass
    def pop(self):
        try:
            res = self.arr[-1]
            self.arr = self.arr[:-1]
        except:
            return []
        return res
    def __str__(self):
        return str(self.arr)
    def __len__(self):
        return len(self.arr)
    def __getitem__(self,i):
        return self.arr[i]
    def __setitem__(self,i,val):
        self.arr[i] = val
    def __contains__(self,val):
        return val in self.arr
    def __iter__(self):
        return self.arr.__iter__()

class stack(container):
    def push(self,*elem):
        self.arr += list(elem)
        
    
class queue(container):
    def push(self,*elem):
        self.arr = list(elem)+ self.arr
        
class graph:
    def __init__(self,names = {},weight = None):
        if type(names) == '__main__.graph':
            self.names = copy.copy(names.names)
        else:
            self.names = copy.copy(names)
        if len(weight) == 0:
            self.weight =np.zeros((len(self.names),len(self.names)))
        else:
            self.weight = weight
    def __str__(self):
        return self.names.__str__()
    def __len__(self):
        return len(self.names)
    def tops(self):
        return [i for i in self.names.keys()]
    def reorder(self,newtop):
        '''
        newtop - имя нового корня графа из числа тех вершин, которые уже в графе
        '''
        keys = self.tops()
        if not newtop in keys:
            return
        d = {newtop:self.names[newtop]}
        for key in keys:
            if key != newtop:
                d[key] = self.names[key]
        self.names = copy.copy(d)
    
    def reachpeaks(self,top):
        if top in self.names.keys():
            self.reorder(top)
            return [i for i in self.walk()]
        return []
    
    def isavailablefrom(self,top1,top2):
        return top2 in self.reachpeaks(top1)
    
    def walk(self,x = None,kind = 'w'):
        if kind == 'w':
            Q = queue()
        else:
            Q = stack()
        keys = [k for k in self.names.keys()]
        if not x: 
            x = {k:0 for k in keys}
            beg = keys[0]
        else:
            for k in x.keys():
                if x[k] == 0:
                    beg = k
                    break
        
        x[beg] = 1
        Q.push(beg)
        while len(Q) != 0:
            u = Q.pop()
            yield u
            for neighbor in self.names[u]:
                if x[neighbor] == 0:
                    Q.push(neighbor)
                    x[neighbor] = 1
        self.x = x

def fullgraph(n = 3):
    #генерация полного графа
    def sub(a,val):
        a = copy.copy(a)
        a.remove(val)
        return a
    keys = [str(i) for i in range(n)]
    d = {key:sub(keys,key) for key in keys}
    return d

def inc_to_list(m):
    #преобразует матрицу смежности в список смежности
    d = {}
    for i in range(len(m)):
        for j in range(len(m)):
            if m[i][j] != 0:
                try:
                    d[str(i+1)].append(str(j+1))
                except:
                    d[str(i+1)] = [str(j+1)]
    return d

def edges(gr):
    '''
    Вынимает из неориентированного графа все ребра
    и сортирует их по возрастанию веса
    '''
    n = gr.weight.shape[0]   
    res = []
    for i in range(n):
        for j in range(i,n):
            if gr.weight[i,j] != 0:
                res.append((gr.weight[i,j],i,j))
    res = np.array(res,dtype = [('weights',float),('froms',int),('tos',int)])
    return np.sort(res,order = 'weights',kind = 'mergesort')

## Реализация алгоритмов:

In [23]:
def Kraskala(gr):
    '''
    O(E*logE + EV)
    Алгоритм Краскала поиска минимального остовного дерева в графе
    возвращает минимальный остов
    
    '''
    i = 1
    n =len(gr.weight)
    # T - это весовая матрица смежности. gr.weight - тоже
    T = np.zeros((n,n))
    # создание списка ребер, отсортированного по весу
    edgs = edges(gr) # O(m*log(m))
    # нулевой шаг алгоритма
    T[edgs[0][1],edgs[0][2]] = edgs[0][0]
    T[edgs[0][2],edgs[0][1]] = edgs[0][0]
    for e in edgs[1:]: 
        # O(nm) - общая сложность на присоединение n-1 ребра
        # e[0] - вес ребра, е[1] - начальная вершина ребра, е[2] - конечная вершина ребра
        if i > n-1: break
        else: i += 1
        #проверка на то, доступна ли из одной вершины другая. Если доступна, то образуется цикл, а нам такие не нужны.
        fl = graph(inc_to_list(T),T).isavailablefrom(str(e[1]+1),str(e[2]+1))        
        if not fl:
            T[e[1],e[2]] = e[0]
            T[e[2],e[1]] = e[0]
    return graph(inc_to_list(T),T)

## Примеры графов для тестов:

In [20]:
# Пример 6.1 стр. 61
g_6_1 = {'1':['2','6','5'],
         '2':['1','3','5'],
         '3':['2','4'],
         '4':['3','5','6'],
         '5':['1','4','6','2'],
         '6':['1','4','5']}
g_6_1_weight= np.array([[0,10,0,0,9,8],
                        [10,0,2,0,6,0],
                        [0,2,0,5,0,0],
                        [0,0,5,0,4,3],
                        [9,6,0,4,0,1],
                        [8,0,0,3,1,0]])


# Пример 6.4 стр. 65
g_6_4 = {'1':['2','3','4'],
         '2':['1','3','11'],
         '3':['1','2','6','7','8'],
         '4':['1','5','6'],
         '5':['4','6','7'],
         '6':['3','4','5','7'],
         '7':['3','5','6','8','9'],
         '8':['3','7','9','10','11'],
         '9':['7','8','10'],
         '10':['8','9','11'],
         '11':['2','8','10']}

g_6_4_weight = np.array([[0,5,3,10,0,0,0,0,0,0,0],
                         [5,0,10,0,0,0,0,0,0,0,5],
                         [3,10,0,0,0,8,10,2,0,0,0],
                         [10,0,0,0,1,1,0,0,0,0,0],
                         [0,0,0,1,0,1,2,0,0,0,0],
                         [0,0,8,1,1,0,10,0,0,0,0],
                         [0,0,10,0,2,10,0,2,1,0,0],
                         [0,0,2,0,0,0,2,0,3,1,7],
                         [0,0,0,0,0,0,1,3,0,1,0],
                         [0,0,0,0,0,0,0,1,1,0,6],
                         [0,5,0,0,0,0,0,7,0,6,0]])



# Пример 6.5 стр. 66
g_6_5 = {'1':['2','4','5'],
        '2':['1','3','4','5','7'],
        '3':['2','9','8'],
        '4':['1','2','6','7','11'],
        '5':['1','2','6','7'],
        '6':['4','5','11'],
        '7':['4','2','5','8','9','10','11'],
        '8':['3','7','9','12'],
        '9':['3','7','8','10'],
        '10':['7','9','12'],
        '11':['4','6','7','12'],
        '12':['8','10','11']}


g_6_5_weight = np.array([[0,6,0,11,5,0,0,0,0,0,0,0],
                      [6,0,15,18,8,0,11,0,0,0,0,0],
                      [0,15,0,0,0,0,0,18,6,0,0,0],
                      [11,18,0,0,0,10,7,0,0,0,17,0],
                      [5,8,0,0,0,15,9,0,0,0,0,0],
                      [0,0,0,10,15,0,0,0,0,0,3,0],
                      [0,11,0,7,9,0,0,9,4,13,12,0],
                      [0,0,18,0,0,0,9,0,14,0,0,5],
                      [0,0,6,0,0,0,4,14,0,19,0,0],
                      [0,0,0,0,0,0,13,0,19,0,0,2],
                      [0,0,0,17,0,3,12,0,0,0,0,7],
                      [0,0,0,0,0,0,0,5,0,2,7,0]])



# Пример 6.6 стр. 67
g_6_6 = fullgraph(7)

g_6_6_weight = np.array([[0,4,3,10,6,7,7],
                        [4,0,5,4,4,3,9],
                        [3,5,0,1,3,2,8],
                        [10,4,1,0,10,8,3],
                        [6,4,3,10,0,4,5],
                        [7,3,2,8,4,0,9],
                        [7,9,8,3,5,9,0]])

graph_66 = graph(g_6_6, g_6_6_weight)
graph_65 = graph(g_6_5, g_6_5_weight)
graph_64 = graph(g_6_4, g_6_4_weight)
graph_61 = graph(g_6_1, g_6_1_weight)

In [27]:
ostov_61 = Kraskala(graph_61)
ostov_64 = Kraskala(graph_64)
ostov_65 = Kraskala(graph_65)
ostov_66 = Kraskala(graph_66)

# Задача: схема мелиорации рисовых полей(далее выведен минимальный остов)

In [25]:
print(ostov_61)

{'1': ['6'], '2': ['3'], '3': ['2', '4'], '4': ['3', '6'], '5': ['6'], '6': ['1', '4', '5']}


# Иные примеры минимальных остовов для примеров графов из методички

In [28]:
print(ostov_64)

{'1': ['2', '3'], '2': ['1', '11'], '3': ['1', '8'], '4': ['5', '6'], '5': ['4', '7'], '6': ['4'], '7': ['5', '9'], '8': ['3', '10'], '9': ['7', '10'], '10': ['8', '9'], '11': ['2']}


In [29]:
print(ostov_65)

{'1': ['2', '5'], '2': ['1'], '3': ['9'], '4': ['7'], '5': ['1', '7'], '6': ['11'], '7': ['4', '5', '8', '9'], '8': ['7', '12'], '9': ['3', '7'], '10': ['12'], '11': ['6', '12'], '12': ['8', '10', '11']}


In [30]:
print(ostov_66)

{'1': ['3'], '2': ['6'], '3': ['1', '4', '5', '6'], '4': ['3', '7'], '5': ['3'], '6': ['2', '3'], '7': ['4']}
